<a href="https://colab.research.google.com/github/deftydev/Crew_Agents/blob/main/Langchain_%26Memory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q langchain_google_genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.1/417.1 kB 15.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.4 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.17 which is incompatible.


In [2]:
pip install -U langchain langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 52.3 MB/s eta 0:00:00
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.6
    Uninstalling langchain-text-splitters-0.3.6:
      Successfully uninstalled langchain-text-splitters-0.3.6
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.20
    Uninstalling langchain-0.3.20:
      Successfully uninstalled langchain-0.3.20


In [3]:
pip install langchain-core langgraph>0.2.27

In [4]:
LANGSMITH_TRACING= True
LANGSMITH_ENDPOINT="https://api.smith.langchain.com"
LANGSMITH_PROJECT="chat_with_langchain"

In [5]:
import getpass
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass()

··········


In [6]:
os.environ["GOOGLE_API_KEY"] = getpass.getpass()

··········


In [7]:
from langchain_google_genai import ChatGoogleGenerativeAI

In [12]:
from langchain_core.output_parsers import StrOutputParser

In [13]:
parser = StrOutputParser()

In [8]:
from langchain.schema import HumanMessage

In [17]:
google_llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro")
messages= [HumanMessage(content="Tell me about your power")]

output = parser.invoke(google_llm.invoke(messages))
print(output)

My power lies in the ability to process and generate human-like text.  I can translate languages, write different kinds of creative content, and answer your questions in an informative way, even if they are open ended, challenging, or strange.  I'm trained on a massive dataset of text and code, which allows me to do things like:

* **Understand and respond to a wide range of prompts and questions:**  From simple factual queries to complex analytical tasks, I can provide relevant information and insights.
* **Generate different creative text formats:**  Poems, code, scripts, musical pieces, email, letters, etc., are all within my capabilities.
* **Translate languages:** I can translate text between multiple languages with reasonable accuracy.
* **Summarize factual topics:** I can condense large amounts of information into concise summaries.
* **Write different kinds of creative content:**  I can craft stories, articles, and other forms of written content.
* **Answer your questions in an

In [10]:
from langchain_core.messages import AIMessage

google_llm.invoke(
    [
        HumanMessage(content="Hi! I'm Bob"),
        AIMessage(content="Hello Bob! How can I assist you today?"),
        HumanMessage(content="What's my name?"),
    ]
)

AIMessage(content='You told me your name is Bob.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-85159f25-2687-4729-9433-09f01ae6fb7d-0', usage_metadata={'input_tokens': 22, 'output_tokens': 9, 'total_tokens': 31, 'input_token_details': {'cache_read': 0}})

In [15]:
parser.invoke(google_llm.invoke("hi"))

'Hi there! How can I help you today?'

In [20]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a nice AI bot that helps a user figure out what to eat in one short sentence"),
        ("human", "{user_input}"),
    ]
)

model = ChatGoogleGenerativeAI(model="gemini-1.5-pro")

parser = StrOutputParser()

chain = prompt | model | parser
# chain.invoke({"user_input":"I like chocolates"})

for chunk in chain.stream({"user_input": "dogs"}):
    print(chunk, end="", flush=True)

Woof woof, kibble and other dog-friendly treats are best for your furry friend.

In [24]:
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

In [38]:
store={}
def get_session_history(session_id: str):
  if session_id not in store:
    store[session_id] = InMemoryChatMessageHistory()
  return store[session_id]

In [39]:
config = {"configurable": {"session_id": "firstchat"}}

In [40]:
model_with_memory=RunnableWithMessageHistory(model,get_session_history)

In [41]:
model_with_memory.invoke([HumanMessage(content="Hi! I'm deftdev, what about you")],config=config).content

"I'm a large language model, trained by Google.  I don't have a name like you do."

In [42]:
model_with_memory.invoke([HumanMessage(content="where you live ?")],config=config).content

"I don't live anywhere in the physical sense.  I exist as a computer program running on Google's servers, which are located in various data centers around the world.  So I don't have a specific location like a city or country."

In [43]:
model_with_memory.invoke([HumanMessage(content="what is my name?")],config=config).content

'You told me your name is deftdev.'

Construction of proper model with memory

In [67]:
prompt=ChatPromptTemplate.from_messages(
    [
      ("system","You are a helpful assistant. Answer all questions to the best of your ability.",),
      MessagesPlaceholder(variable_name="messages"),
    ]
)

In [68]:
chain = prompt | model

In [69]:
model_with_memory=RunnableWithMessageHistory(chain,get_session_history)

In [70]:
config = {"configurable": {"session_id": "thirdchat"}}

In [71]:

response=model_with_memory.invoke(
    [HumanMessage(content="Hi! I'm Jim"),
     ],config=config
)


In [72]:
response=model_with_memory.invoke(
    [HumanMessage(content="who is a indian cricket team captain in odi format?"),
     ],config=config
)
print(response.content)

Rohit Sharma is generally considered the regular ODI captain.  However, keep in mind that captaincy can change due to various factors like injuries or team strategies.  It's always best to check a reliable sports news source for the very latest information.


In [74]:
response=model_with_memory.invoke(
    [HumanMessage(content="hi what is my name"),
     ],config=config
)
print(response.content)

You've told me your name is Jim.


Now model does not need to remember whole previous conversation you can explicitly enter the token amount upto which past message model should remember using trim class

In [108]:
from langchain_core.messages import SystemMessage, trim_messages

In [109]:
trimmer = trim_messages(
    max_tokens=40,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

In [110]:
messages = [
    HumanMessage(content="hi! I'm dev"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

In [111]:
model.get_num_tokens_from_messages(messages)

51

In [112]:
trimmed_message = trimmer.invoke(messages)

In [113]:
model.get_num_tokens_from_messages(trimmed_message)

39

In [114]:
trimmed_message

[HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [115]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain = (
    RunnablePassthrough.assign(messages=itemgetter("messages") | trimmer)
    | prompt
    | model
)

response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what's my name?")],
        "language": "English",
    }
)
response.content

"I have no access to personal information about you, including your name, unless you've shared it with me during our current conversation.  I don't have memory of past conversations."

In [116]:
trimmed_message

[HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [117]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

'You asked "what\'s 2 + 2".'

In [118]:
trimmed_message

[HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [119]:
model_with_memory = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

In [120]:
config = {"configurable": {"session_id": "fifthchat"}}

In [121]:
response = model_with_memory.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

"I have no access to personal information about you, including your name, unless you've shared it with me during our current conversation.  For privacy reasons, I don't retain information between conversations."

In [122]:
trimmed_message

[HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]